In [1]:
import sys
import os
from pathlib import Path

current_dir = str(Path(os.getcwd()).parent.parent)
print(f"Current dir: {current_dir}")
sys.path.insert(0, current_dir)

Current dir: d:\Programming\Python\comfyui-image-scorer


In [2]:
import numpy as np
from step03training.full_data.data_utils import (
    load_training_data,
    filter_unused_features,
    add_interaction_features,
    load_training_scores,
)

from shared.paths import vectors_file, scores_file, filtered_data

print("--- Step 1: Loading Data ---")
if not os.path.exists(filtered_data):
    x, y = load_training_data(vectors_file, scores_file)
else:
    x = np.array([])
    y = load_training_scores(scores_file)


print(f"Loaded Data Shape: {x.shape}")

print("\n--- Step 2: Filtering Unused Features ---")
# Removes features with zero variance or zero importance in a quick probe
x, kept_indices = filter_unused_features(x, y)
print(f"Filtered Data Shape: {x.shape}")

print("\n--- Step 3: Generating Interaction Features ---")
# Adds top 500 polynomial interactions (feature_A * feature_B)
x, _ = add_interaction_features(x, y, target_k=200)
print(f"Final Data Shape (with Interactions): {x.shape}")
print("Data Preparation Complete.")

prepare_dir: D:\Programming\Python\comfyui-image-scorer\step02prepare
--- Step 1: Loading Data ---
Loaded Data Shape: (0,)

--- Step 2: Filtering Unused Features ---
Filtering features... Initial shape: (0,)
Loading filtered data from cache: D:\Programming\Python\comfyui-image-scorer\step03training\output\filtered_data.npz
Filtered Data Shape: (8218, 2057)

--- Step 3: Generating Interaction Features ---
Loading interaction data from cache: D:\Programming\Python\comfyui-image-scorer\step03training\output\interaction_data.npz
Final Data Shape (with Interactions): (8218, 2257)
Data Preparation Complete.


In [3]:
import random
from IPython.display import clear_output
from shared.config import config
from step03training.full_data.run import optimize_hyperparameters
from step03training.full_data.config_utils import (
    crossover_config,
    generate_random_config,
    #generate_slowest_setup
)

# Setup Loop Variables
current_cfg = generate_random_config()
top_cfg = config["training"]["top"]
fastest_cfg = config["training"]["fastest"]
slowest_cfg = config["training"]["slowest"]
#slowest_cfg=generate_slowest_setup()
#config["training"]["slowest"]= slowest_cfg
max_iters = config["training"]["max_iters"]
max_combos = config["training"]["max_combos"]
print(f"Initialized HPO loop with max_iters={max_iters}, max_combos={max_combos}")

for i in range(max_iters):
    clear_output(wait=True)
    # Refresh references
    top_cfg = config["training"]["top"]
    fastest_cfg = config["training"]["fastest"]
    slowest_cfg = config["training"]["slowest"]

    max_combos = config["training"]["max_combos"]

    # Strategy selection
    rand_val = random.random()
    if rand_val < 0.05:
        base_cfg = current_cfg
        strategy = "RANDOM_START"
    elif rand_val < 0.25:
        base_cfg = fastest_cfg
        strategy = "FASTEST"
    elif rand_val < 0.60:
        base_cfg = top_cfg
        strategy = "TOP"
    elif rand_val < 0.75:
        # Crossover
        candidates = [
            c for c in [top_cfg, fastest_cfg, slowest_cfg] if c["best_score"] > -9999
        ]
        if len(candidates) < 2:
            candidates = [c for c in [top_cfg, fastest_cfg, current_cfg] if c]
            if len(candidates) < 2:
                candidates = [generate_random_config(), generate_random_config()]

        parents = random.sample(candidates, 2)
        base_cfg = crossover_config(dict(parents[0]), dict(parents[1]))
        strategy = "CROSSOVER"
    else:
        base_cfg = slowest_cfg
        strategy = "SLOWEST"
        # max_combos = 1

    print(f"\nIter {i + 1}/{max_iters} — Strategy: {strategy}")

    # optimize_hyperparameters handles updates and saving internally now.
    results = optimize_hyperparameters(
        base_cfg=base_cfg, max_combos=max_combos, X=x, y=y, strategy=strategy
    )

    # Info only loop
    for candidate_cfg, metrics in results:
        r2 = metrics["r2"]
        t_time = metrics["training_time"]

        # Check if this result matches current bests (just for log/info)
        is_top = r2 == top_cfg["best_score"]
        is_fastest = t_time == fastest_cfg["training_time"]

        if is_top:
            print(f"  [Info] This batch produced the current TOP score: {r2:.6f}")
        if is_fastest:
            print(
                f"  [Info] This batch produced the current FASTEST time: {t_time:.4f}s"
            )

    # Refresh local current_cfg for next RANDOM_START or Crossover inheritance
    if results:
        # Simply take the last one as 'current' for random drift
        current_cfg = results[-1][0]

print(f"\nFinished optimization.")
print(f"Top R2: {top_cfg['best_score']:.6f}")
print(
    f"Fastest Time: {fastest_cfg['training_time']:.6f}s (R2: {fastest_cfg['best_score']:.6f})"
)
print(
    f"Slowest (High Score) Time: {slowest_cfg['training_time']:.6f}s (R2: {slowest_cfg['best_score']:.6f})"
)


Iter 26/100 — Strategy: SLOWEST
Optimizing hyperparameters over grid: {'learning_rate': [0.0013310000000000002, 0.0012100000000000001, 0.0010890000000000001], 'n_estimators': [1782, 1620, 1458], 'num_leaves': [988, 899, 809], 'max_depth': [96, 88, 79], 'min_child_samples': [12, 11, 9], 'reg_alpha': [0.06788100000000002, 0.061710000000000015, 0.05553900000000001], 'reg_lambda': [0.6134445900000002, 0.5576769000000001, 0.5019092100000001], 'subsample': [0.3060090000000001, 0.27819000000000005, 0.25037100000000007], 'colsample_bytree': [0.41490900000000014, 0.3771900000000001, 0.3394710000000001], 'min_split_gain': [0.016105100000000004, 0.014641000000000003, 0.013176900000000004], 'early_stopping_rounds': [178, 162, 145]}
Evaluating hyperparameter combo 1/3, with params: {'learning_rate': 0.0010890000000000001, 'n_estimators': 1782, 'num_leaves': 988, 'max_depth': 96, 'min_child_samples': 11, 'reg_alpha': 0.06788100000000002, 'reg_lambda': 0.5019092100000001, 'subsample': 0.306009000000

Training LightGBM:   5%|▌         | 97/1782 [00:31<08:52,  3.16it/s]

KeyboardInterrupt: 